In [80]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from itertools import product
from scipy.optimize import minimize
from numba import njit

segment_count_beam = 50
vertical_level_count = 300
length_beam = 1000 # mm
youngs_modulus = 2e3 # MPa
moment_of_inertia = 20 # mm^4
channel_width_half = 10 # mm
E_list = []
F_list = []
u_list = []

if vertical_level_count % 2 == 0:
    vertical_level_count += 1

if segment_count_beam > length_beam/(2*channel_width_half):
    previous = segment_count_beam
    segment_count_beam = int(length_beam/(2*channel_width_half))

y_levels = np.linspace(-channel_width_half, channel_width_half, vertical_level_count)
s, ds = np.linspace(0, length_beam, segment_count_beam+1, retstep=True)

@njit
def segment_energy_functional(I_val, J_val, K_val, horizontal_force):
    theta1 = np.arcsin((J_val - I_val) / ds)
    theta2 = np.arcsin((K_val - J_val) / ds)

    internal_strain_energy = 0.5 * youngs_modulus * moment_of_inertia * ( (theta2 - theta1)/ds )**2 * ds
    external_force_work = horizontal_force * np.cos(theta1) * ds
    
    return internal_strain_energy + external_force_work


@njit
def viterbi_algorithm(vertical_level_count, segment_count_beam, segment_energy_functional, horizontal_force):
    # C and survivor arrays now initialized with -1 to signify "not set" or "infinity" for costs
    # Using 3D arrays (I, J, m)
    C = np.full((vertical_level_count, vertical_level_count, segment_count_beam + 1), np.inf, dtype=np.float64)
    survivor = np.full((vertical_level_count, vertical_level_count, segment_count_beam + 1), -1, dtype=np.int32)

    index_zero = vertical_level_count // 2

    # Initialization for m = 1
    for I in range(vertical_level_count):
        for J in range(vertical_level_count):
            if I == index_zero:
                C[I, J, 1] = 0.0
                survivor[I, J, 1] = -1 # No predecessor for the first segment
            else:
                C[I, J, 1] = np.inf

    # Main Viterbi loop
    for m in range(2, segment_count_beam + 1):
        for J in range(vertical_level_count):
            for K in range(vertical_level_count):
                best_cost = np.inf
                best_prev_I = -1

                for I in range(vertical_level_count):
                    prev_cost = C[I, J, m - 1]

                    if prev_cost == np.inf:
                        continue

                    trans_cost = segment_energy_functional(y_levels[I], y_levels[J], y_levels[K], horizontal_force)
                    total_cost = prev_cost + trans_cost

                    if total_cost < best_cost:
                        best_cost = total_cost
                        best_prev_I = I
                
                C[J, K, m] = best_cost
                survivor[J, K, m] = best_prev_I

    # Traceback - Find the minimum cost path end
    min_cost = np.inf
    best_final_J = -1
    best_final_K = -1

    for J in range(vertical_level_count):
        for K in range(vertical_level_count):
            if K == index_zero: # Path must end at index_zero
                cost = C[J, K, segment_count_beam]
                if cost < min_cost:
                    min_cost = cost
                    best_final_J = J
                    best_final_K = K

    if best_final_J == -1 or min_cost == np.inf:
        # Return an empty NumPy array of appropriate dtype if no valid path
        return np.empty(0, dtype=y_levels.dtype) 
    else:
        # Use a Numba-compatible list for temporary storage of indices
        # We can append integers to it.
        reconstructed_y_indices_list = []
        current_J_trace = best_final_J
        current_K_trace = best_final_K

        # Add the last two indices
        reconstructed_y_indices_list.append(current_K_trace)
        reconstructed_y_indices_list.append(current_J_trace)

        # Trace back through the survivor array
        for m_trace in range(segment_count_beam, 1, -1):
            prev_I_trace = survivor[current_J_trace, current_K_trace, m_trace]
            if prev_I_trace == -1: # Indicates no valid predecessor found
                break
            reconstructed_y_indices_list.append(prev_I_trace)
            current_K_trace = current_J_trace
            current_J_trace = prev_I_trace

        reconstructed_y_indices_list.reverse()
        
        # Convert indices to y_levels values and return as a NumPy array
        # Ensure y_levels is a NumPy array when passed into the jitted function
        y_viterbi = np.empty(len(reconstructed_y_indices_list), dtype=y_levels.dtype)
        for i in range(len(reconstructed_y_indices_list)):
            y_viterbi[i] = y_levels[reconstructed_y_indices_list[i]]
            
        return y_viterbi

@njit
def x_from_y(y):
    dy = np.diff(y)
    dx = np.sqrt(ds**2 - dy**2)
    x = np.cumsum(dx)
    # For include_initial=True, we need to manually add an initial x-value.
    # Assuming the first x value is 0. If it's something else, adjust accordingly.
    return np.concatenate((np.array([0.0]), x))

@njit
def energy_functional(y, horizontal_force):
    theta = np.arcsin(np.diff(y)/ds)
    diffs = np.diff(theta)
    v = 0.5 * youngs_modulus * moment_of_inertia * np.sum( (diffs/ds)**2 ) * ds + horizontal_force * np.sum(np.cos(theta)) * ds
    return v

In [ ]:
force_values = np.arange(0.3, 30, 0.05)

for horizontal_force in force_values:  
    y_viterbi = viterbi_algorithm(vertical_level_count, segment_count_beam, segment_energy_functional, horizontal_force)
    x_viterbi = x_from_y(y_viterbi)

    res = minimize( energy_functional, 
                    x0=y_viterbi, 
                    constraints = [{"type": "eq", "fun": lambda y: y[-1]},
                                {"type": "eq", "fun": lambda y: y[0]}],
                    bounds = [(-channel_width_half, channel_width_half)
                            for _ in range(segment_count_beam+1)],
                    method="SLSQP",
                    args=(horizontal_force)
                    )

    y_minimized = res.x
    x_minimized = x_from_y(y_minimized)

    E_list.append(res.fun)
    F_list.append(horizontal_force)
    u_list.append(x_minimized[-1]-length_beam)